In [2]:
from datasets import load_dataset
import random
import json

random.seed(10)

/Users/sumersao/miniconda3/envs/twitbits/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
"""
Called by clean. Listify source documents, clean up summary (remove "– " at the beginning)

input: single datapoint {'document': String, 'summary': String}
output: {'document': List, 'summary': String}
"""
def clean_single_dp(datapoint):
  docs_str = datapoint['document']
  doc_list = docs_str.split("|||||") #list of the source documents

  sum = datapoint['summary']
  summary_clean = sum[2:] #get rid of "– " at beginning of each summary

  return doc_list, summary_clean

In [4]:
"""
Clean each datapoint (listify docs, clean summaries) 
Create dicitonary where keys are number of source docs, in case we decide to 
aggregate our data based on #docs (to standardize input size)

input: unaugmented multiNews dataset: List of {'document': String, 'summary': String}
output: 
  all_data: List of {'document_list': List, 'clean_summary': String}
  numdocs_dict: Dictionary where key = #source docs, value = list of datapoints - 
                each one is {'document': String, 'summary': String}
"""
def clean_orig(data): #takes all data
  all_data = []

  for point in data:
    #augment single point
    doc_list, summary_clean = clean_single_dp(point)

    #add to all_data
    all_data += [{'document_list': doc_list, 'clean_summary': summary_clean}]

  return all_data

In [5]:
"""
Called by group_data. Aggregate given list of datapoints.

input: list of datapoints, each of which is {document_list: [...], clean_summary: "..."}
output: 
  combo_documents: list of doc lists
  combo_summaries_str: String of combined summaries separated by \n\n
  combo_summaries_list: list of summaries
"""
def combine_points(data_list):
  combo_documents = [] #list of lists
  combo_summaries_list = [] #list of strings

  for point in data_list:
    combo_documents += point['document_list']
    combo_summaries_list += [point['clean_summary']]

  combo_summaries_str = "\n\n".join(combo_summaries_list) #string (concatenated summaries separated by a black line)
  # we also want to add a "- " to the beginning of each summary
  combo_summaries_str = "- " + combo_summaries_str

  return combo_documents, combo_summaries_str, combo_summaries_list

In [6]:
"""
Randomly partition dataset into groups of GROUP_SIZE, aggregate each group.

input: full cleaned dataset - List of {'document_list': List, 'clean_summary': String}
output: List of aggregated data {'documents': List, 'summary': String'
"""
def group_data(data):
  GROUP_SIZE = 3

  #partition
  random.shuffle(data)
  groups = [data[i:i+GROUP_SIZE] for i in range(0, len(data), GROUP_SIZE)]

  #if last group is a lonely datapoint, merge it with the previous group
  if len(groups[-1]) == 1:
    groups[-2] += groups[-1]
    groups = groups[:-1]

  #aggregate each group
  for i in range(len(groups)):
    combo_docs, combo_sum_str, _ = combine_points(groups[i])
    aggregated_group = {'documents': combo_docs, 'summary': combo_sum_str}
    groups[i] = aggregated_group

  return groups

In [7]:
"""
Compute distribution of #documents i.e. how many datapoints have x docs

input: grouped and aggregated dataset - List of {'documents': List, 'summary': String'
output: {int: int} key = number of documents, value = number of datapoints with that number of documents
"""
def get_group_numdoc_freq(grouped_data):
    numdoc_frequency = {}

    for data in grouped_data:
        num_docs = len(data.get('documents', []))
        if num_docs in numdoc_frequency:
            numdoc_frequency[num_docs] += 1
        else:
            numdoc_frequency[num_docs] = 1

    return numdoc_frequency


"""
Full data augmentation process
"""
def augment_data(data):
  clean_data = clean_orig(data)
  augmented_data = group_data(clean_data)
  return augmented_data #List of {'documents': List, 'summary': String}

In [8]:
dataset= load_dataset('multi_news')

train = list(dataset['train'])
val = list(dataset['validation'])
test = list(dataset['test'])

print("Train: ", len(train))
print("Val: ", len(val))
print("Test: ", len(test))

Found cached dataset multi_news (/Users/sumersao/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)
100%|██████████| 3/3 [00:00<00:00, 126.33it/s]


Train:  44972
Val:  5622
Test:  5622


In [9]:
augmented_test = augment_data(test)
numdoc_freq = get_group_numdoc_freq(augmented_test)

print("Number of Documents Frequency:")
for numdocs, frequency in numdoc_freq.items():
    print(f"#documents {numdocs}: {frequency}")

Number of Documents Frequency:
#documents 9: 287
#documents 7: 471
#documents 6: 298
#documents 8: 429
#documents 10: 154
#documents 11: 106
#documents 5: 22
#documents 14: 20
#documents 13: 34
#documents 12: 40
#documents 15: 6
#documents 19: 1
#documents 16: 4
#documents 18: 1
#documents 17: 1


In [15]:
# save the augmented test set
with open('augmented_test.json', 'w') as f:
    json.dump(augmented_test, f)